In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

def cstruct(event,results):
    global Q
    global Qs

    thread=event['Thread']
    index=event['Index']
       
    if index==1:
        return ''
    for i in range(index-1,0,-1):
        temp=Q[0][(Q[0].Thread == thread) & (Q[0].Index==i)].iloc[0]
        if temp['Event']=='send':
            results.append(temp['ID'])
            event=temp
            results.extend(cstruct(event,results))
        else:
            results.append(temp['ID'])
            send=Qs[0].loc[temp.name]
            results.append(send['ID'])
            results.extend(cstruct(send,results))
    return list(set(results))
    
def no_index(start,finish,t,event,R):
    #if type(event)==int:
    #    event=Qr[(Qr.ID==R[finish])].iloc[0]
    for l in range(start,finish):
        if t[l]>0 and R[l] in cstruct(event,[]):
            return True
        else:
            continue
    return False

    

def construct_race_table(Q,Qs,Qr,race_set):
    R=[] #create race_set
    D=[] #number of race_set
    heading=[] #{r1,r2,....r}

    R.append('')
    D.append('')
    heading.append('dummy')
    # set R,D
    for key in race_set.keys():
        if len(race_set[key])>0 and Q[Q['ID']==key].iloc[0].coler=='white':
            R.append(key) # raceのあるイベントを追加
            D.append(len(race_set[key])) #raceの数を追加
            heading.append(key) # r
    table=pd.DataFrame([],columns=heading)

    t=np.zeros(len(R)) #raceの数分の配列 #t[0]はダミー

    while True:

        results=[]

        max_index=''
        for i in range(len(t)-1,0,-1):
            if t[i]<D[i] and t[i]!=-1:
                max_index=i
                break
        if max_index=='':
            break
        t[i]+=1

        if t[i]==1: #just changed t[i] from 0 to 1

            for j in tqdm(range(i+1,len(R))):
                if t[j]!=-1 and (Qr[Qr.ID==R[i]].iloc[0].ID in cstruct(Qr[Qr.ID==R[j]].iloc[0],[])):

                    t[j]=-1

        for j in tqdm(range(i+1,len(R))):
            if(t[j]==D[j]):
                t[j]=0 #just change t[j] from dj to 0

                for k in tqdm(range(j+1,len(R))):
                    if t[k]==-1 and Qr[Qr.ID==R[j]].iloc[0].ID in cstruct(Qr[Qr.ID==R[k]].iloc[0],[]) and no_index(1,k,t,Qr[(Qr.ID==R[k])].iloc[0],R):
                        t[k]=0
        #let s be the t[i] sending event in race_set(ri)
        s= race_set[R[i]][int(t[i])-1] 


        if no_index(1,len(R)-1,t,Qs[(Qs.ID==s)].iloc[0],R)==False:
            table=table.append(pd.Series(t,index=table.columns),ignore_index=True)
    return table.drop('dummy',axis=1)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 250.62it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


,r1,r3
0,0.0,1.0
1,1.0,0.0
2,1.0,1.0


In [51]:

def remove_race(race1,race2): #race_set(r,Q)=race_set(r,Q)-race_set(r,V)
    for key1 in race1.keys():
        try:
            dup=list(set(race1[key1] + race2[key1]))
            print(dup)
            for index in range(len(dup)):
                race1[key1].remove(dup[index])
        except Exception as e:
            print(e)
            continue
    return race1


variants=''
V=[]
#variants=GenerateVariants(Q,V0)

race_set[0]={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}
race_set[1]={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}

#while !variants:
    

In [100]:
#============ここから定義============
    
heading_res=('Thread','Port','Index')
heading_snd=('Thread','Port','Event','Index')

Qs=[pd.DataFrame({})]
Qr=[pd.DataFrame({})]
#Q is SYN-sequence
Qr[0]=pd.read_csv('SYN-res.csv',names=heading_res)
Qs[0]=pd.read_csv('SYN-snd.csv',names=heading_snd)


r_list=list(range(1,len(Qr[0])+1))
r_list=list(map(lambda x:'r'+str(x),r_list)) #['r1', 'r2', 'r3', 'r4']
s_list=list(range(1,len(Qs[0])+1))
s_list=list(map(lambda x:'s'+str(x),s_list)) #['s1', 's2', 's3', 's4']

Qr[0].insert(0,'ID',r_list) #attach the name of event
Qs[0].insert(0,'ID',s_list) #attach the name of event

Qr[0]['coler']='white'

Q=[pd.DataFrame({})] #Q is SYN-sequence
Q[0]=pd.merge(Qr[0],Qs[0],how='outer')
race_set[0]={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}
race_set[1]={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]}

#============ここまで定義============

In [90]:
#def Generatevariants(Q,number):
#race_set[0]=remove_race(race_set[0],race_set[0])
table=construct_race_table(Q[0],Qs,Qr,race_set[0])
columns=list(table.columns)
for i in range(0,len(table)):
    Q.append(Q[i]) #新しいテーブルを作成
    for key in range(0,len(columns)):
        if table.iloc[i][columns[key]]>0: #race_set の交換
            #Q[i+1]のテーブルを修正
            change_event=Q[i+1][Q[i+1]['ID']==columns[key]].iloc[0].ID # receive
            new_partner=race_set[0][Q[i+1][Q[i+1]['ID']==columns[key]].iloc[0].ID][int(table.iloc[i][columns[key]])-1] #send
            pre_partner=
            Q[i+1][Q[i+1]['ID']==columns[key]]
            table.iloc[i][columns[key]]

0it [00:00, ?it/s]
0it [00:00, ?it/s]
  0%|          | 0/1 [00:00<?, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:00<00:00, 200.32it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]


In [102]:
Qr[0]

,ID,Thread,Port,Index,coler
0,r1,T2,{p2},1,white
1,r2,T2,{p2},2,white
2,r3,T3,{p3},1,white
3,r4,T3,{p3},2,white


In [103]:
Qs[0]

,ID,Thread,Port,Event,Index
0,s1,T1,{p2},send,1
1,s2,T4,{p2},send,1
2,s3,T2,{p3},send,3
3,s4,T4,{p3},send,2


In [93]:
Q[i+1][Q[i+1]['ID']==columns[key]].iloc[0].ID # これのペアを
race_set[0][Q[i+1][Q[i+1]['ID']==columns[key]].iloc[0].ID][int(table.iloc[i][columns[key]])-1] # これをペアに変更

'r3'

In [98]:
Q[i+1][Q[i+1]['ID']==columns[key]].iloc[0].ID

'r3'

In [53]:
remove_race(race_set1,race_set2)

['s2']
[]
['s4']
[]


{'r1': [], 'r2': [], 'r3': [], 'r4': []}

In [37]:
race=list(set(race_set1['r1'] + race_set2['r1']))

In [43]:
Q

[   ID Thread  Port  Index  coler Event
 0  r1     T2  {p2}      1  white   NaN
 1  r2     T2  {p2}      2  white   NaN
 2  r3     T3  {p3}      1  white   NaN
 3  r4     T3  {p3}      2  white   NaN
 4  s1     T1  {p2}      1    NaN  send
 5  s2     T4  {p2}      1    NaN  send
 6  s3     T2  {p3}      3    NaN  send
 7  s4     T4  {p3}      2    NaN  send]